In [1]:
import csv, json
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import gensim
from nltk.corpus import stopwords
stopset = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize


In [2]:
post_feat = pd.read_csv('./top_200.gzip', compression='gzip', header=0)

In [3]:
sub_list = np.unique(post_feat['subreddit'].tolist())
if 'conspiracy' in sub_list:
    print "true"

true


In [4]:
print len(sub_list)

38146


In [5]:
raw = []
for sub in sub_list:
    #print sub
    corpus = post_feat.loc[post_feat['subreddit']==sub, 'title']
    corp = " ".join(map(str, corpus))
    corp = re.sub('[^A-Za-z .-]+', '', corp)
    corp = corp.replace('-', '')
    corp = corp.replace('…', '')
    corp = corp.replace('.', '')
    corp = corp.replace('...', '')
    corp = corp.replace('..x', '')

    raw.append(corp.decode('ascii', 'ignore').encode('ascii'))


In [6]:
print "Number of documents", len(raw)

Number of documents 38146


In [7]:
gen_docs = [[w.lower() for w in word_tokenize(text) if not w in stopwords.words('english')] 
            for text in raw]
#print(gen_docs)

In [8]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary[5])
print(dictionary.token2id['road'])
print("Number of words in dictionary:",len(dictionary))
#for i in range(len(dictionary)):
#    print(i, dictionary[i])

adventures
2703
('Number of words in dictionary:', 958121)


In [9]:
rcorpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
#print(rcorpus)

In [10]:
tf_idf = gensim.models.TfidfModel(rcorpus)
print(tf_idf)
s = 0
for i in rcorpus:
    s += len(i)
print(s)

TfidfModel(num_docs=38146, num_nnz=18341690)
18341690


In [11]:
sims = gensim.similarities.Similarity('./',tf_idf[rcorpus],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

Similarity index with 38146 documents in 1 shards (stored under ./)
<class 'gensim.similarities.docsim.Similarity'>


In [12]:
len(sims)

38146

In [13]:
consp_ind = list(sub_list).index('conspiracy')

In [14]:
print consp_ind

24229


In [15]:
for item in list(enumerate(sims)):
    if item[0]==consp_ind:
        consp_sim= item[1]

In [16]:
indices = np.argsort(consp_sim)

In [17]:
top_20 = []


wfile = open('similarity_score.csv', 'w')

for i in range(1, len(indices)):
    top_20.append(indices[len(indices)-i])
    
for i in range(len(indices)-1):
    #print i
    towrite = sub_list[top_20[i]] + ',' + str(consp_sim[top_20[i]]) + '\n'
    wfile.write(towrite)
    
wfile.close()
